# Population aged 15-74 (LFS) by region, labour status, observations and quarter

## Data source
[Open interactive table](https://www.statistikdatabasen.scb.se/pxweb/en/ssd/START__AM__AM0401__AM0401N/NAKUBefolkningLK/table/tableViewLayout1/)

## What this notebook does
- Fetches unemployment rates for all Swedish counties for the latest available year from SCB (PXWeb).
- Converts the response into a tidy `pandas` DataFrame.
- Saves the result to `data/UnempDirect.csv`.


In [15]:
# Imports
from pathlib import Path  # file system paths

import pandas as pd  # data wrangling
from pyscbwrapper import SCB  # SCB PXWeb API wrapper

In [16]:
# Paths

ROOT = Path.cwd().resolve().parents[1]

# Output directory for exported files
data_dir = ROOT / "data"
data_dir.mkdir(parents=True, exist_ok=True)

## SCB table and variables

### Table ID
The `TABLE` tuple identifies the PXWeb table to query.

### Variables
`scb.info()` / `scb.get_variables()` list available regions, years, and observations.


In [17]:
# SCB PXWeb table identifier
TABLE = ("en", "AM", "AM0401", "AM0401N", "NAKUBefolkningLK")

In [18]:
# Initialize a client for this table
scb = SCB(*TABLE)

In [19]:
# Inspect table metadata (dimensions, available values)
scb.info()

{'title': 'Population aged 15-74 (LFS) by region, labour status, sex, observations and quarter',
 'variables': [{'code': 'Region',
   'text': 'region',
   'values': ['00',
    '0050',
    '01',
    '0180',
    '03',
    '04',
    '05',
    '06',
    '07',
    '08',
    '09',
    '10',
    '12',
    '1280',
    '13',
    '14',
    '1480',
    '17',
    '18',
    '19',
    '20',
    '21',
    '22',
    '23',
    '24',
    '25'],
   'valueTexts': ['Sweden',
    'The whole country excluding Stockholm, Göteborg and Malmö',
    'Stockholm county',
    'Stockholm',
    'Uppsala county',
    'Södermanland county',
    'Östergötland county',
    'Jönköping county',
    'Kronoberg county',
    'Kalmar county',
    'Gotland county',
    'Blekinge county',
    'Skåne county',
    'Malmö',
    'Halland county',
    'Västra Götaland county',
    'Göteborg',
    'Värmland county',
    'Örebro county',
    'Västmanland county',
    'Dalarna county',
    'Gävleborg county',
    'Västernorrland county',

In [20]:
# Fetch variable values as a dict (used to build filters)
var_ = scb.get_variables()

## Build the query

### Filters
- Regions: counties only
- Quarter: 2025K1, (optional to latest quarter)
- Observation: Unemployment rate and Margin of Error


In [21]:
## ====== Counties ======
counties_key = next(k for k in var_ if "region" in k.lower())

counties_obs = [r for r in var_["region"] if "county" in r.lower()]

## ===== Quarter ======

quarter_key = next(k for k in var_ if "quarter" in k.lower())

quarter_obs = ["2025K1"]

# helper for sorting and picking up the latest.

# def parse_quarter(q):
#     year, quarter = q.split("K")
#     return int(year), int(quarter)

# latest_quarter = max(var_["quarter"], key=parse_quarter)

## ===== Labor = unemployed, 

labour_key = next(k for k in var_ if "labour" in k.lower())
labour_key_clean = labour_key.replace(" ", "") 

labour_obs = ["unemployed"]


In [22]:
## ========  Choose the observations to display

observation_key = next(k for k in var_ if "observation" in k.lower())
observation =  var_[observation_key][2:4]


In [23]:
# Build query (match exact variable names from `scb.info()` / `scb.get_variables()`)

scb.set_query(
    **{
        counties_key: counties_obs,
        quarter_key: quarter_obs,
        labour_key_clean: labour_obs,
        observation_key: observation
    }
)

In [24]:
# Execute query and extract the observations list
scb_data = scb.get_data()
scb_fetch = scb_data["data"]

In [25]:
# Map region codes (e.g., "01") to readable county names
codes = scb.get_query()["query"][0]["selection"]["values"]

counties_dict = dict(zip(codes, counties_obs))

## Transform response
Convert the PXWeb response into a tidy table with one row per county (for the selected quarter).

### Output columns
- `code`: region code
- `county`: county name
- `quarter`: observation quarter
- `rate`: Percent
- `margin_of_error`: Margin of error percent



In [26]:
# Convert the SCB response to a tidy DataFrame (one row per county)

records = []

for r in scb_fetch:
    # r["key"] contains [region_code, year, ...] depending on the table
    code, labor_status, quarter = r["key"][:3]
    name = counties_dict.get(code, code)
    rate, margin_of_error = r["values"][:2]
    records.append({
        "code": code,
        "county": name,
        "labor": labor_status,
        "year_quarter": quarter,
        "rate": rate,
        "margin_of_error": margin_of_error
    })

df = pd.DataFrame(records)

## Preview and export
Preview a few rows, then write the final CSV.


In [27]:
# Preview the final table
df.head(10)

,code,county,labor,year_quarter,rate,margin_of_error
0,01,Stockholm county,ALÖS,2025K1,8.7,0.9
1,03,Uppsala county,ALÖS,2025K1,11.6,3.3
2,04,Södermanland county,ALÖS,2025K1,10.6,3.9
3,05,Östergötland county,ALÖS,2025K1,8.5,2.2
4,06,Jönköping county,ALÖS,2025K1,7.8,2.5
5,07,Kronoberg county,ALÖS,2025K1,10.8,4.1
6,08,Kalmar county,ALÖS,2025K1,10.7,3.7
7,09,Gotland county,ALÖS,2025K1,..,..
8,10,Blekinge county,ALÖS,2025K1,12.9,3.6
9,12,Skåne county,ALÖS,2025K1,10.9,1.5


In [28]:
# Export to CSV
df.to_csv(data_dir / "UnempDirect.csv", index=False)

## Output
The resulting file is saved as `data/UnempDirect.csv`.
